In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [8]:
df=pd.read_csv('train.csv')
x=df.drop(['Survived','PassengerId','Name',"Ticket"],axis=1)
y=df['Survived']

In [9]:
print(x)
print(y)

     Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0         3    male  22.0      1      0   7.2500   NaN        S
1         1  female  38.0      1      0  71.2833   C85        C
2         3  female  26.0      0      0   7.9250   NaN        S
3         1  female  35.0      1      0  53.1000  C123        S
4         3    male  35.0      0      0   8.0500   NaN        S
..      ...     ...   ...    ...    ...      ...   ...      ...
886       2    male  27.0      0      0  13.0000   NaN        S
887       1  female  19.0      0      0  30.0000   B42        S
888       3  female   NaN      1      2  23.4500   NaN        S
889       1    male  26.0      0      0  30.0000  C148        C
890       3    male  32.0      0      0   7.7500   NaN        Q

[891 rows x 8 columns]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [20]:
from sklearn.model_selection import train_test_split
x_train,x_testing,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [59]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

numerical_cols=x.select_dtypes(include=['int64','float64']).columns
categorical_cols=x.select_dtypes(include=['object']).columns

Onehot_pipeline=Pipeline([
    ('Imputing',SimpleImputer(missing_values=np.nan,strategy='most_frequent')),
    ('encoding',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

numerical_pipeline=Pipeline([
    ('Imputing',SimpleImputer(missing_values=np.nan,strategy='mean')),
    ('Scaling',StandardScaler())
])

preprocessing=ColumnTransformer( transformers=
    [('Imputing numerical',numerical_pipeline,numerical_cols),
     ('Imputing categorical data',Onehot_pipeline,categorical_cols)
    ]
)

model=Pipeline(
    [
        ('Preprocessing',preprocessing),
        ('classification',XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42, use_label_encoder=False, eval_metric='logloss'))
    ]
)

model.fit(x_train,y_train)

,steps,"[('Preprocessing', ...), ('classification', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('Imputing numerical', ...), ('Imputing categorical data', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [60]:
y_predicted=model.predict(x_testing)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy=accuracy_score(y_test,y_predicted)
com=confusion_matrix(y_test,y_predicted)
print(accuracy)
print(com)

0.8379888268156425
[[104   6]
 [ 23  46]]


In [61]:
x_test=pd.read_csv('test.csv')
test_df=x_test.drop(['Name','Ticket','PassengerId'],axis=1)

y_pred=model.predict(test_df)

submission=pd.DataFrame(
    {
        'PassengerId':x_test['PassengerId'],
        'Survived':y_pred
    }
)

submission.to_csv('Submission.csv',index=False)
print("Saved")

Saved
